## Seminar 1: Fun with Word Embeddings (3 points)

Today we gonna play with word embeddings: train our own little embedding, load one from   gensim model zoo and use it to visualize text corpora.

This whole thing is gonna happen on top of embedding dataset.

__Requirements:__  `pip install --upgrade nltk gensim bokeh` , but only if you're running locally.

In [6]:
# download the data:
#!wget https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1 -O ./quora.txt
# alternative download link: https://yadi.sk/i/BPQrUu1NaTduEw

In [7]:
#!curl -o quora.txt https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1

In [70]:
import numpy as np

with open("./quora.txt", encoding="utf-8") as file:
    data = list(file)

data[50]

"What TV shows or books help you read people's body language?\n"

__Tokenization:__ a typical first step for an nlp task is to split raw data into words.
The text we're working with is in raw format: with all the punctuation and smiles attached to some words, so a simple str.split won't do.

Let's use __`nltk`__ - a library that handles many nlp tasks like tokenization, stemming or part-of-speech tagging.

In [10]:
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

print(tokenizer.tokenize(data[50]))

['What', 'TV', 'shows', 'or', 'books', 'help', 'you', 'read', 'people', "'", 's', 'body', 'language', '?']


In [11]:
# TASK: lowercase everything and extract tokens with tokenizer. 
# data_tok should be a list of lists of tokens for each line in data.

data_tok =[tokenizer.tokenize(line.lower()) for line in data]

In [13]:
assert all(isinstance(row, (list, tuple)) for row in data_tok), "please convert each line into a list of tokens (strings)"
assert all(all(isinstance(tok, str) for tok in row) for row in data_tok), "please convert each line into a list of tokens (strings)"
is_latin = lambda tok: all('a' <= x.lower() <= 'z' for x in tok)
assert all(map(lambda l: not is_latin(l) or l.islower(), map(' '.join, data_tok))), "please make sure to lowercase the data"

In [14]:
print([' '.join(row) for row in data_tok[:2]])

["can i get back with my ex even though she is pregnant with another guy ' s baby ?", 'what are some ways to overcome a fast food addiction ?']


__Word vectors:__ as the saying goes, there's more than one way to train word embeddings. There's Word2Vec and GloVe with different objective functions. Then there's fasttext that uses character-level models to train word embeddings. 

The choice is huge, so let's start someplace small: __gensim__ is another nlp library that features many vector-based models incuding word2vec.

In [15]:
from gensim.models import Word2Vec
model = Word2Vec(data_tok, 
                 vector_size=32,      # embedding vector size
                 min_count=5,  # consider words that occured at least 5 times
                 window=5).wv  # define context as a 5-word window around the target word
# wv - word vectors

In [17]:
# now you can get word vectors !
model.get_vector('anything')

array([-1.1314794 , -0.98807895,  0.96032697,  1.6752628 ,  2.3355324 ,
        2.0666993 ,  1.1964502 , -4.268514  ,  0.8450309 ,  2.6495779 ,
       -0.5686738 ,  1.0088397 ,  2.9306414 ,  0.6695664 ,  3.5096085 ,
       -1.1256049 ,  1.0142395 , -0.609636  ,  0.94691247, -3.4163678 ,
       -3.3491352 ,  0.79071957, -2.803897  , -1.4293475 ,  2.8251247 ,
       -4.1628137 , -0.27094066,  1.1671027 ,  0.7364791 ,  0.0119278 ,
       -1.1700269 , -0.78652817], dtype=float32)

In [26]:
# or query similar words directly. Go play with it!
model.most_similar('fate')

[('destiny', 0.8367618918418884),
 ('aristotle', 0.8313214778900146),
 ('patriotism', 0.8136816024780273),
 ('philosopher', 0.8130513429641724),
 ('communism', 0.8125616908073425),
 ('capitalism', 0.8111039400100708),
 ('nationalism', 0.8066451549530029),
 ('possession', 0.8039574027061462),
 ('spirit', 0.8018816709518433),
 ('illuminati', 0.7961613535881042)]

### Using pre-trained model

Took it a while, huh? Now imagine training life-sized (100~300D) word embeddings on gigabytes of text: wikipedia articles or twitter posts. 

Thankfully, nowadays you can get a pre-trained word embedding model in 2 lines of code (no sms required, promise).

In [27]:
import gensim.downloader as api
model = api.load('glove-twitter-100')

[==================================================] 100.0% 387.1/387.1MB downloaded


In [63]:
model.most_similar(positive=['fish', 'camel'], negative=["water"])

[('donkey', 0.6421257257461548),
 ('snake', 0.6102703213691711),
 ('lobster', 0.6028012633323669),
 ('pig', 0.5982781052589417),
 ('ostrich', 0.594143807888031),
 ('tails', 0.5850961208343506),
 ('squid', 0.583195686340332),
 ('turtle', 0.5822569131851196),
 ('horse', 0.5794443488121033),
 ('hairy', 0.5791943669319153)]

### Visualizing word vectors

One way to see if our vectors are any good is to plot them. Thing is, those vectors are in 30D+ space and we humans are more used to 2-3D.

Luckily, we machine learners know about __dimensionality reduction__ methods.

Let's use that to plot 1000 most frequent words

In [64]:
words = model.index_to_key[:1000] 

print(words[::100])

['<user>', '_', 'please', 'apa', 'justin', 'text', 'hari', 'playing', 'once', 'sei']


In [66]:
# for each word, compute it's vector with model
word_vectors = np.array([model.get_vector(x) for x in words])

In [69]:
assert isinstance(word_vectors, np.ndarray)
assert word_vectors.shape == (len(words), 100)
assert np.isfinite(word_vectors).all()

#### Linear projection: PCA

The simplest linear dimensionality reduction method is __P__rincipial __C__omponent __A__nalysis.

In geometric terms, PCA tries to find axes along which most of the variance occurs. The "natural" axes, if you wish.

<img src="https://github.com/yandexdataschool/Practical_RL/raw/master/yet_another_week/_resource/pca_fish.png" style="width:30%">


Under the hood, it attempts to decompose object-feature matrix $X$ into two smaller matrices: $W$ and $\hat W$ minimizing _mean squared error_:

$$\|(X W) \hat{W} - X\|^2_2 \to_{W, \hat{W}} \min$$
- $X \in \mathbb{R}^{n \times m}$ - object matrix (**centered**);
- $W \in \mathbb{R}^{m \times d}$ - matrix of direct transformation;
- $\hat{W} \in \mathbb{R}^{d \times m}$ - matrix of reverse transformation;
- $n$ samples, $m$ original dimensions and $d$ target dimensions;



In [86]:
from sklearn.decomposition import PCA

# map word vectors onto 2d plane with PCA. Use good old sklearn api (fit, transform)
# after that, normalize vectors to make sure they have zero mean and unit variance
word_vectors_pca = PCA(n_components=2).fit_transform(word_vectors)
mean = np.mean(word_vectors_pca, axis=0)
std = np.std(word_vectors_pca, axis=0)
word_vectors_pca = (word_vectors_pca - mean) / std
# and maybe MORE OF YOUR CODE here :)

In [87]:
assert word_vectors_pca.shape == (len(word_vectors), 2), "there must be a 2d vector for each word"
assert max(abs(word_vectors_pca.mean(0))) < 1e-5, "points must be zero-centered"
assert max(abs(1.0 - word_vectors_pca.std(0))) < 1e-2, "points must have unit variance"

#### Let's draw it!

In [88]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    if isinstance(color, str): color = [color] * len(x)
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

Loading BokehJS ...

In [89]:
draw_vectors(word_vectors_pca[:, 0], word_vectors_pca[:, 1], token=words)

# hover a mouse over there and see if you can identify the clusters

figure(id='p1004', ...)

### Visualizing neighbors with t-SNE
PCA is nice but it's strictly linear and thus only able to capture coarse high-level structure of the data.

If we instead want to focus on keeping neighboring points near, we could use TSNE, which is itself an embedding method. Here you can read __[more on TSNE](https://distill.pub/2016/misread-tsne/)__.

In [92]:
from sklearn.manifold import TSNE

# map word vectors onto 2d plane with TSNE. hint: don't panic it may take a minute or two to fit.
# normalize them as just lke with pca

word_tsne = TSNE(n_components=2).fit_transform(word_vectors)
mean = np.mean(word_tsne, axis=0)
std = np.std(word_tsne, axis=0)
word_tsne = (word_tsne - mean) / std

In [93]:
draw_vectors(word_tsne[:, 0], word_tsne[:, 1], color='green', token=words)

figure(id='p1049', ...)

### Visualizing phrases

Word embeddings can also be used to represent short phrases. The simplest way is to take __an average__ of vectors for all tokens in the phrase with some weights.

This trick is useful to identify what data are you working with: find if there are any outliers, clusters or other artefacts.

Let's try this new hammer on our data!


In [108]:
def get_phrase_embedding(phrase):
    """
    Convert phrase to a vector by aggregating it's word embeddings. See description above.
    """
    # 1. lowercase phrase
    # 2. tokenize phrase
    # 3. average word vectors for all words in tokenized phrase
    # skip words that are not in model's vocabulary
    # if all words are missing from vocabulary, return zeros
    
    vector = np.zeros([model.vector_size], dtype='float32')
    vocab =  model.index_to_key
    # YOUR CODE
    avg_vec = []
    word_tok = tokenizer.tokenize(phrase.lower())
    for word in word_tok:
        if word in vocab:
            avg_vec.append(model.get_vector(word))

    avg_vec = np.array(avg_vec)
    if len(avg_vec) == 0:
        return vector

    return np.mean(avg_vec, axis=0)

In [111]:
vector = get_phrase_embedding("I'm very sure. This never happened to me before...")

assert np.allclose(vector[::10],
                   np.array([ 0.31807372, -0.02558171,  0.0933293 , -0.1002182 , -1.0278689 ,
                             -0.16621883,  0.05083408,  0.17989802,  1.3701859 ,  0.08655966],
                              dtype=np.float32))

In [112]:
# let's only consider ~5k phrases for a first run.
chosen_phrases = data[::len(data) // 1000]

# compute vectors for chosen phrases
phrase_vectors = np.array([get_phrase_embedding(phrase) for phrase in chosen_phrases])

In [113]:
assert isinstance(phrase_vectors, np.ndarray) and np.isfinite(phrase_vectors).all()
assert phrase_vectors.shape == (len(chosen_phrases), model.vector_size)

In [114]:
# map vectors into 2d space with pca, tsne or your other method of choice
# don't forget to normalize

phrase_vectors_2d = TSNE(n_components=2).fit_transform(phrase_vectors)

phrase_vectors_2d = (phrase_vectors_2d - phrase_vectors_2d.mean(axis=0)) / phrase_vectors_2d.std(axis=0)

In [115]:
draw_vectors(phrase_vectors_2d[:, 0], phrase_vectors_2d[:, 1],
             phrase=[phrase[:50] for phrase in chosen_phrases],
             radius=20,)

figure(id='p1094', ...)

Finally, let's build a simple "similar question" engine with phrase embeddings we've built.

In [121]:
from tqdm.auto import tqdm 

In [127]:
# compute vector embedding for all lines in data
#data_vectors = np.array([get_phrase_embedding(l) for l in data])
data_vectors = []
for i in tqdm(range(len(data))):
    data_vectors.append(get_phrase_embedding(data[i]))
data_vectors = np.array(data_vectors)

100%|████████████████████████████████████████████████████████████████████████████████████| 537272/537272 [1:29:12<00:00, 100.37it/s]


In [128]:
np.save('data_vectors', data_vectors)

In [118]:
from scipy.spatial.distance import cosine

In [198]:
def find_nearest(query, k=10):
    """
    given text line (query), return k most similar lines from data, sorted from most to least similar
    similarity should be measured as cosine between query and line embedding vectors
    hint: it's okay to use global variables: data and data_vectors. see also: np.argpartition, np.argsort
    """
    # YOUR CODE
    query_vec = get_phrase_embedding(query)
    dists = np.array([cosine(query_vec, phrase) for phrase in data_vectors])
    # k_idx = np.argsort(dists)[:-k]
    sorted_idx = np.argsort(dists)
    k_idx = []
    for idx in sorted_idx:
        if (data_vectors[idx] == np.zeros([model.vector_size], dtype='float32')).all():
            continue
        k_idx.append(idx)
        if len(k_idx) == k:
            break
    print(k_idx)
    return np.array(data)[k_idx]
    #return <YOUR CODE: top-k lines starting from most similar>

In [199]:
results = find_nearest(query="How do i enter the matrix?", k=10)

[348630, 194211, 535949, 248982, 416639, 358896, 384831, 499334, 277540, 78818]


In [200]:
results[3]

'What can I do to save the world?\n'

In [201]:
results = find_nearest(query="How do i enter the matrix?", k=10)

print(''.join(results))

assert len(results) == 10 and isinstance(results[0], str)
assert results[0] == 'How do I get to the dark web?\n'
assert results[3] == 'What can I do to save the world?\n'

[348630, 194211, 535949, 248982, 416639, 358896, 384831, 499334, 277540, 78818]
How do I get to the dark web?
What should I do to enter hollywood?
How do I use the Greenify app?
What can I do to save the world?
How do I win this?
How do I think out of the box? How do I learn to think out of the box?
How do I find the 5th dimension?
How do I use the pad in MMA?
How do I estimate the competition?
What do I do to enter the line of event management?



In [202]:
r = find_nearest(query="How does Trump?", k=10)
print(''.join(r))

[536175, 9618, 210436, 37557, 18780, 187158, 518962, 203826, 110684, 45478]
What does Donald Trump think about Israel?
What books does Donald Trump like?
What does Donald Trump think of India?
What does India think of Donald Trump?
What does Donald Trump think of China?
What does Donald Trump think about Pakistan?
What companies does Donald Trump own?
What does Dushka Zapata think about Donald Trump?
How does it feel to date Ivanka Trump?
What does salesforce mean?



In [203]:
r = find_nearest(query="Why don't i ask a question myself?", k=10)
print(''.join(r))

[481169, 289957, 421503, 323721, 374877, 468871, 153603, 478070, 10739, 194652]
Why don't I get a date?
Why do you always answer a question with a question? I don't, or do I?
Why can't I ask a question anonymously?
Why don't I get a girlfriend?
Why don't I have a boyfriend?
I don't have no question?
Why can't I take a joke?
Why don't I ever get a girl?
Can I ask a girl out that I don't know?
Why don't I have a girlfriend?



__Now what?__
* Try running TSNE on all data, not just 1000 phrases
* See what other embeddings are there in the model zoo: `gensim.downloader.info()`
* Take a look at [FastText](https://github.com/facebookresearch/fastText) embeddings
* Optimize find_nearest with locality-sensitive hashing: use [nearpy](https://github.com/pixelogik/NearPy) or `sklearn.neighbors`.